In [1]:
from dotenv import load_dotenv
import os
import pandas as pd
import csv
import re

load_dotenv()
CH_HOST = os.getenv('CH_HOST')
CH_PORT = os.getenv('CH_PORT') 
CH_USER = os.getenv('CH_USER')
CH_PASSWORD = os.getenv('CH_PASSWORD')

import clickhouse_connect
client = clickhouse_connect.get_client(host=CH_HOST, port=CH_PORT, user=CH_USER, password=CH_PASSWORD)



In [2]:
df = pd.read_csv(r'../data/HI-Large_Trans.csv')
df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/08/01 00:17,20,800104D70,20.0,800104D70,6794.63,US Dollar,6794.63,US Dollar,Reinvestment,0.0
1,2022/08/01 00:02,3196,800107150,3196.0,800107150,7739.29,US Dollar,7739.29,US Dollar,Reinvestment,0.0
2,2022/08/01 00:17,1208,80010E430,1208.0,80010E430,1880.23,US Dollar,1880.23,US Dollar,Reinvestment,0.0
3,2022/08/01 00:03,1208,80010E650,20.0,80010E6F0,73966883.00,US Dollar,73966883.00,US Dollar,Cheque,0.0
4,2022/08/01 00:02,1208,80010E650,20.0,80010EA30,45868454.00,US Dollar,45868454.00,US Dollar,Cheque,0.0


In [3]:
df.columns

Index(['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1',
       'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format', 'Is Laundering'],
      dtype='object')

In [4]:
df.rename(columns={'Account.1': 'Account_1'}, inplace=True)

Same thing
df = df.rename(columns={'Account.1': 'Account_1'})
df.rename(columns={'Account.1': 'Account_1'}, inplace=True)

In [16]:
print(df.dtypes)


Timestamp             datetime64[ns]
From Bank                      int64
Account                       object
To Bank                      float64
Account.1                     object
Amount Received              float64
Receiving Currency            object
Amount Paid                  float64
Payment Currency              object
Payment Format                object
Is Laundering                float64
dtype: object


In [9]:
df.isna().sum()

Timestamp             0
From Bank             0
Account               0
To Bank               1
Account_1             1
Amount Received       1
Receiving Currency    1
Amount Paid           1
Payment Currency      1
Payment Format        1
Is Laundering         1
dtype: int64

In [17]:
df['Timestamp'] = df['Timestamp'].astype(str)
# df['Account.1'] = df['Account.1'].astype(str)
# df['Receiving Currency'] = df['Receiving Currency'].astype(str)
# df['Payment Currency'] = df['Payment Currency'].astype(str)
# df['Payment Format'] = df['Payment Format'].astype(str)


In [ ]:
client.command('DROP TABLE  if exists card_data.hi_large ')
client.command('''CREATE TABLE card_data.hi_large (
                                    Timestamp String
                                    ,`From Bank` Int64
                                    ,Account String
                                    ,`To Bank` Nullable(Float32)
                                    ,`Account_1` Nullable(String)
                                    ,`Amount Received` Nullable(Float64)
                                    ,`Receiving Currency` Nullable(String)
                                    ,`Amount Paid` Nullable(Float64)
                                   ,`Payment Currency` Nullable(String)
                                    ,`Payment Format` Nullable(String)
                                    ,`Is Laundering` Nullable(Float64)
) ENGINE = MergeTree()
    ORDER BY Timestamp''')

In [10]:
client.insert('card_data.hi_large', df)

In [2]:
df_small= pd.read_csv(r'../data/HI-Small_Trans.csv')
df_small.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


In [3]:
df_medium= pd.read_csv(r'../data/LI-Medium_Trans.csv')
df_medium.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:15,20,800104D70,20,800104D70,8095.07,US Dollar,8095.07,US Dollar,Reinvestment,0
1,2022/09/01 00:18,3196,800107150,3196,800107150,7739.29,US Dollar,7739.29,US Dollar,Reinvestment,0
2,2022/09/01 00:23,1208,80010E430,1208,80010E430,2654.22,US Dollar,2654.22,US Dollar,Reinvestment,0
3,2022/09/01 00:19,3203,80010EA80,3203,80010EA80,13284.41,US Dollar,13284.41,US Dollar,Reinvestment,0
4,2022/09/01 00:27,20,800104D20,20,800104D20,9.72,US Dollar,9.72,US Dollar,Reinvestment,0


In [4]:
df_hi_medium= pd.read_csv(r'../data/HI-Medium_Trans.csv')
df_hi_medium.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:17,20,800104D70,20,800104D70,6794.63,US Dollar,6794.63,US Dollar,Reinvestment,0
1,2022/09/01 00:02,3196,800107150,3196,800107150,7739.29,US Dollar,7739.29,US Dollar,Reinvestment,0
2,2022/09/01 00:17,1208,80010E430,1208,80010E430,1880.23,US Dollar,1880.23,US Dollar,Reinvestment,0
3,2022/09/01 00:03,1208,80010E650,20,80010E6F0,73966883.00,US Dollar,73966883.00,US Dollar,Cheque,0
4,2022/09/01 00:02,1208,80010E650,20,80010EA30,45868454.00,US Dollar,45868454.00,US Dollar,Cheque,0


In [5]:
df_medium.columns

Index(['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1',
       'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format', 'Is Laundering'],
      dtype='object')

In [5]:
df_medium.rename(columns={'Account.1': 'Account_1'}, inplace=True)
df_hi_medium.rename(columns={'Account.1': 'Account_1'}, inplace=True)
df_small.rename(columns={'Account.1': 'Account_1'}, inplace=True)

In [10]:
client.insert('card_data.hi_small', df_small)

In [6]:
df_combined_medium= pd.concat([df_medium, df_hi_medium], ignore_index=True)

print(df_combined_medium)

                 Timestamp  From Bank    Account  To Bank  Account_1  \
0         2022/09/01 00:15         20  800104D70       20  800104D70   
1         2022/09/01 00:18       3196  800107150     3196  800107150   
2         2022/09/01 00:23       1208  80010E430     1208  80010E430   
3         2022/09/01 00:19       3203  80010EA80     3203  80010EA80   
4         2022/09/01 00:27         20  800104D20       20  800104D20   
...                    ...        ...        ...      ...        ...   
63149716  2022/09/16 23:37     215117  84F680E11   223744  8521DCB81   
63149717  2022/09/16 23:59    1209449  84E4F43D1      123  8521E3901   
63149718  2022/09/16 23:47     214637  8521E8FF1      114  8521E8941   
63149719  2022/09/16 23:38        114  8521E9E41      114  8521E8941   
63149720  2022/09/16 23:34     220769  8519C3511      114  8521E8941   

          Amount Received Receiving Currency   Amount Paid Payment Currency  \
0             8095.070000          US Dollar   8095.0700

In [7]:
client.insert('card_data.medium', df_combined_medium)

In [2]:
def parse_patterns_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    
    data = []
    anomaly_type = None
    
    for line in lines:
        line = line.strip()
        if line.startswith("BEGIN LAUNDERING ATTEMPT"):
            anomaly_type = line.split("-")[-1].strip()
        elif line.startswith("END LAUNDERING ATTEMPT"):
            anomaly_type = None
        elif anomaly_type and ',' in line:
            row = line.split(',')
            row.append(anomaly_type)
            data.append(row)
    
    columns = [
        "Timestamp", "From Bank", "Account", "To Bank", "Account_1",
        "Amount Received", "Receiving Currency", "Amount Paid",
        "Payment Currency", "Payment Format", "Is Laundering", "Anomaly Type"
    ]
    patterns_large = pd.DataFrame(data, columns=columns)
    
    patterns_large["From Bank"] = patterns_large["From Bank"].astype(int)
    patterns_large["To Bank"] = pd.to_numeric(patterns_large["To Bank"], errors="coerce")
    patterns_large["Amount Received"] = pd.to_numeric(patterns_large["Amount Received"], errors="coerce")
    patterns_large["Amount Paid"] = pd.to_numeric(patterns_large["Amount Paid"], errors="coerce")
    patterns_large["Is Laundering"] = pd.to_numeric(patterns_large["Is Laundering"], errors="coerce")
    
    return patterns_large

file_path = '../data/HI-Large_Patterns.txt'
patterns_large = parse_patterns_file(file_path)

print(patterns_large.head())

          Timestamp  From Bank    Account  To Bank  Account_1  \
0  2022/08/09 05:14        952  8139F54E0   111632  8062C56E0   
1  2022/08/13 13:09     111632  8062C56E0     8456  81363F620   
2  2022/08/15 07:40     118693  823D5EB90    13729  801CF2E60   
3  2022/08/15 14:19      13729  801CF2E60   123621  81A7090F0   
4  2022/08/13 12:40      24750  81363F410   213834  808757B00   

   Amount Received Receiving Currency  Amount Paid Payment Currency  \
0          5331.44          US Dollar      5331.44        US Dollar   
1          5602.59          US Dollar      5602.59        US Dollar   
2          1400.54          US Dollar      1400.54        US Dollar   
3          1467.94          US Dollar      1467.94        US Dollar   
4         16898.29          US Dollar     16898.29        US Dollar   

  Payment Format  Is Laundering Anomaly Type  
0            ACH              1        STACK  
1            ACH              1        STACK  
2            ACH              1        ST

In [ ]:
client.insert('card_data.large_patterns', patterns_large)

In [5]:
patterns_large

,Timestamp,From Bank,Account,To Bank,Account_1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Anomaly Type
0,2022/08/09 05:14,952,8139F54E0,111632,8062C56E0,5331.44,US Dollar,5331.44,US Dollar,ACH,1,STACK
1,2022/08/13 13:09,111632,8062C56E0,8456,81363F620,5602.59,US Dollar,5602.59,US Dollar,ACH,1,STACK
2,2022/08/15 07:40,118693,823D5EB90,13729,801CF2E60,1400.54,US Dollar,1400.54,US Dollar,ACH,1,STACK
3,2022/08/15 14:19,13729,801CF2E60,123621,81A7090F0,1467.94,US Dollar,1467.94,US Dollar,ACH,1,STACK
4,2022/08/13 12:40,24750,81363F410,213834,808757B00,16898.29,US Dollar,16898.29,US Dollar,ACH,1,STACK
...,...,...,...,...,...,...,...,...,...,...,...,...
137931,2022/11/08 09:12,1538,8008279E0,12883,801345490,5698.52,Euro,5698.52,Euro,ACH,1,Out
137932,2022/11/20 18:34,1538,8008279E0,1226,8001672F0,6043.82,Euro,6043.82,Euro,ACH,1,Out
137933,2022/11/25 11:27,1538,8008279E0,1213,8006E6420,9483.76,Euro,9483.76,Euro,ACH,1,Out
137934,2022/11/28 09:15,1538,8008279E0,1827,80117E140,11919.51,Euro,11919.51,Euro,ACH,1,Out


In [7]:
patterns_large['Timestamp'] = pd.to_datetime(patterns_large['Timestamp'])

patterns_large['Year'] = patterns_large['Timestamp'].dt.year
patterns_large['Month'] = patterns_large['Timestamp'].dt.month_name()
patterns_large['Day'] = patterns_large['Timestamp'].dt.day
patterns_large['Time'] = patterns_large['Timestamp'].dt.time


In [8]:
patterns_large

,Timestamp,From Bank,Account,To Bank,Account_1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Anomaly Type,Year,Month,Day,Time
0,2022-08-09 05:14:00,952,8139F54E0,111632,8062C56E0,5331.44,US Dollar,5331.44,US Dollar,ACH,1,STACK,2022,August,9,05:14:00
1,2022-08-13 13:09:00,111632,8062C56E0,8456,81363F620,5602.59,US Dollar,5602.59,US Dollar,ACH,1,STACK,2022,August,13,13:09:00
2,2022-08-15 07:40:00,118693,823D5EB90,13729,801CF2E60,1400.54,US Dollar,1400.54,US Dollar,ACH,1,STACK,2022,August,15,07:40:00
3,2022-08-15 14:19:00,13729,801CF2E60,123621,81A7090F0,1467.94,US Dollar,1467.94,US Dollar,ACH,1,STACK,2022,August,15,14:19:00
4,2022-08-13 12:40:00,24750,81363F410,213834,808757B00,16898.29,US Dollar,16898.29,US Dollar,ACH,1,STACK,2022,August,13,12:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137931,2022-11-08 09:12:00,1538,8008279E0,12883,801345490,5698.52,Euro,5698.52,Euro,ACH,1,Out,2022,November,8,09:12:00
137932,2022-11-20 18:34:00,1538,8008279E0,1226,8001672F0,6043.82,Euro,6043.82,Euro,ACH,1,Out,2022,November,20,18:34:00
137933,2022-11-25 11:27:00,1538,8008279E0,1213,8006E6420,9483.76,Euro,9483.76,Euro,ACH,1,Out,2022,November,25,11:27:00
137934,2022-11-28 09:15:00,1538,8008279E0,1827,80117E140,11919.51,Euro,11919.51,Euro,ACH,1,Out,2022,November,28,09:15:00


In [9]:
columns_to_move = ['Year', 'Month', 'Day', 'Time']
new_column_order = columns_to_move + [col for col in patterns_large.columns if col not in columns_to_move]

patterns_large = patterns_large[new_column_order]
patterns_large

,Year,Month,Day,Time,Timestamp,From Bank,Account,To Bank,Account_1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Anomaly Type
0,2022,August,9,05:14:00,2022-08-09 05:14:00,952,8139F54E0,111632,8062C56E0,5331.44,US Dollar,5331.44,US Dollar,ACH,1,STACK
1,2022,August,13,13:09:00,2022-08-13 13:09:00,111632,8062C56E0,8456,81363F620,5602.59,US Dollar,5602.59,US Dollar,ACH,1,STACK
2,2022,August,15,07:40:00,2022-08-15 07:40:00,118693,823D5EB90,13729,801CF2E60,1400.54,US Dollar,1400.54,US Dollar,ACH,1,STACK
3,2022,August,15,14:19:00,2022-08-15 14:19:00,13729,801CF2E60,123621,81A7090F0,1467.94,US Dollar,1467.94,US Dollar,ACH,1,STACK
4,2022,August,13,12:40:00,2022-08-13 12:40:00,24750,81363F410,213834,808757B00,16898.29,US Dollar,16898.29,US Dollar,ACH,1,STACK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137931,2022,November,8,09:12:00,2022-11-08 09:12:00,1538,8008279E0,12883,801345490,5698.52,Euro,5698.52,Euro,ACH,1,Out
137932,2022,November,20,18:34:00,2022-11-20 18:34:00,1538,8008279E0,1226,8001672F0,6043.82,Euro,6043.82,Euro,ACH,1,Out
137933,2022,November,25,11:27:00,2022-11-25 11:27:00,1538,8008279E0,1213,8006E6420,9483.76,Euro,9483.76,Euro,ACH,1,Out
137934,2022,November,28,09:15:00,2022-11-28 09:15:00,1538,8008279E0,1827,80117E140,11919.51,Euro,11919.51,Euro,ACH,1,Out


In [24]:
print(patterns_large.columns)
patterns_large

# patterns_large = patterns_large.drop(columns=['Timestamp'])
# patterns_large

Index(['Year', 'Month', 'Day', 'Time', 'From Bank', 'Account', 'To Bank',
       'Account_1', 'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format', 'Is Laundering', 'Anomaly Type'],
      dtype='object')


,Year,Month,Day,Time,From Bank,Account,To Bank,Account_1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Anomaly Type
0,2022,August,9,05:14:00,952,8139F54E0,111632,8062C56E0,5331.44,US Dollar,5331.44,US Dollar,ACH,1,STACK
1,2022,August,13,13:09:00,111632,8062C56E0,8456,81363F620,5602.59,US Dollar,5602.59,US Dollar,ACH,1,STACK
2,2022,August,15,07:40:00,118693,823D5EB90,13729,801CF2E60,1400.54,US Dollar,1400.54,US Dollar,ACH,1,STACK
3,2022,August,15,14:19:00,13729,801CF2E60,123621,81A7090F0,1467.94,US Dollar,1467.94,US Dollar,ACH,1,STACK
4,2022,August,13,12:40:00,24750,81363F410,213834,808757B00,16898.29,US Dollar,16898.29,US Dollar,ACH,1,STACK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137931,2022,November,8,09:12:00,1538,8008279E0,12883,801345490,5698.52,Euro,5698.52,Euro,ACH,1,Out
137932,2022,November,20,18:34:00,1538,8008279E0,1226,8001672F0,6043.82,Euro,6043.82,Euro,ACH,1,Out
137933,2022,November,25,11:27:00,1538,8008279E0,1213,8006E6420,9483.76,Euro,9483.76,Euro,ACH,1,Out
137934,2022,November,28,09:15:00,1538,8008279E0,1827,80117E140,11919.51,Euro,11919.51,Euro,ACH,1,Out


In [3]:
anomaly_type = patterns_large['Anomaly Type'].unique()
anomaly_type

array(['STACK', 'CYCLE:  Max 12 hops', 'In', 'CYCLE:  Max 5 hops',
       'BIPARTITE', 'CYCLE:  Max 8 hops', 'Out', 'GATHER',
       'CYCLE:  Max 9 hops', 'RANDOM:  Max 3 hops', 'CYCLE:  Max 2 hops',
       'RANDOM:  Max 10 hops', 'RANDOM:  Max 1 hops',
       'RANDOM:  Max 5 hops', 'RANDOM:  Max 11 hops',
       'CYCLE:  Max 11 hops', 'RANDOM:  Max 7 hops', 'CYCLE:  Max 4 hops',
       'RANDOM:  Max 6 hops', 'CYCLE:  Max 6 hops', 'RANDOM:  Max 2 hops',
       'CYCLE:  Max 10 hops', 'RANDOM:  Max 4 hops',
       'CYCLE:  Max 13 hops', 'CYCLE:  Max 3 hops', 'RANDOM:  Max 9 hops',
       'RANDOM:  Max 12 hops', 'RANDOM:  Max 8 hops',
       'CYCLE:  Max 7 hops'], dtype=object)

In [6]:
patterns_large['Anomaly Type']

0         STACK
1         STACK
2         STACK
3         STACK
4         STACK
          ...  
137931      Out
137932      Out
137933      Out
137934      Out
137935      Out
Name: Anomaly Type, Length: 137936, dtype: object

In [7]:
anomaly_counts = patterns_large['Anomaly Type'].value_counts()

unique_anomalies = patterns_large['Anomaly Type'].nunique()
print("Number of unique anomaly types:", unique_anomalies)
print("Frequency of each anomaly type:  ---------> ")
print(anomaly_counts)


Number of unique anomaly types: 29
Frequency of each anomaly type:  ---------> 
Anomaly Type
In                      39648
GATHER                  26978
STACK                   23624
Out                     13554
BIPARTITE               12174
CYCLE:  Max 13 hops      1638
CYCLE:  Max 12 hops      1464
RANDOM:  Max 11 hops     1320
CYCLE:  Max 11 hops      1287
RANDOM:  Max 12 hops     1272
CYCLE:  Max 2 hops       1192
CYCLE:  Max 10 hops      1150
RANDOM:  Max 9 hops      1017
RANDOM:  Max 10 hops     1010
CYCLE:  Max 9 hops       1008
CYCLE:  Max 8 hops        904
CYCLE:  Max 7 hops        868
RANDOM:  Max 8 hops       832
CYCLE:  Max 6 hops        816
RANDOM:  Max 7 hops       805
RANDOM:  Max 6 hops       762
CYCLE:  Max 5 hops        705
CYCLE:  Max 3 hops        672
CYCLE:  Max 4 hops        636
RANDOM:  Max 1 hops       605
RANDOM:  Max 5 hops       585
RANDOM:  Max 4 hops       576
RANDOM:  Max 2 hops       426
RANDOM:  Max 3 hops       408
Name: count, dtype: int64


In [8]:
filtered_data = patterns_large[patterns_large['Receiving Currency'] == 'US Dollar']
filtered_data

,Timestamp,From Bank,Account,To Bank,Account_1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Anomaly Type
0,2022/08/09 05:14,952,8139F54E0,111632,8062C56E0,5331.44,US Dollar,5331.44,US Dollar,ACH,1,STACK
1,2022/08/13 13:09,111632,8062C56E0,8456,81363F620,5602.59,US Dollar,5602.59,US Dollar,ACH,1,STACK
2,2022/08/15 07:40,118693,823D5EB90,13729,801CF2E60,1400.54,US Dollar,1400.54,US Dollar,ACH,1,STACK
3,2022/08/15 14:19,13729,801CF2E60,123621,81A7090F0,1467.94,US Dollar,1467.94,US Dollar,ACH,1,STACK
4,2022/08/13 12:40,24750,81363F410,213834,808757B00,16898.29,US Dollar,16898.29,US Dollar,ACH,1,STACK
...,...,...,...,...,...,...,...,...,...,...,...,...
137924,2022/11/12 05:52,18252,807CAB760,13729,8095ED810,3467.53,US Dollar,3467.53,US Dollar,ACH,1,CYCLE: Max 9 hops
137925,2022/11/19 11:34,13729,8095ED810,211755,8063140E0,3210.86,US Dollar,3210.86,US Dollar,ACH,1,CYCLE: Max 9 hops
137926,2022/11/20 17:55,211755,8063140E0,8456,80999EE00,3332.99,US Dollar,3332.99,US Dollar,ACH,1,CYCLE: Max 9 hops
137930,2022/11/05 23:59,1538,8008279E0,23435,8017863F0,16425.62,US Dollar,16425.62,US Dollar,ACH,1,Out


In [6]:
filtered_data = patterns_large[patterns_large['Receiving Currency'] == 'Euro']
filtered_data

,Timestamp,From Bank,Account,To Bank,Account_1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Anomaly Type
8,2022/08/03 13:28,119211,814AB4F60,132965,81B88A230,14567.69,Euro,14567.69,Euro,ACH,1,CYCLE: Max 12 hops
10,2022/08/11 07:16,137089,810C71940,216618,81D5302D0,14567.69,Euro,14567.69,Euro,ACH,1,CYCLE: Max 12 hops
11,2022/08/13 05:09,216618,81D5302D0,24083,81836B520,13629.75,Euro,13629.75,Euro,ACH,1,CYCLE: Max 12 hops
16,2022/08/27 07:10,7818,8037732C0,121523,80D1BD2F0,10636.75,Euro,10636.75,Euro,ACH,1,CYCLE: Max 12 hops
29,2022/08/19 12:06,18500,8096A9720,122120,80C6A8560,77206819.93,Euro,77206819.93,Euro,ACH,1,CYCLE: Max 5 hops
...,...,...,...,...,...,...,...,...,...,...,...,...
137928,2022/12/02 06:52,239461,81047FC40,224391,816047D00,2753.12,Euro,2753.12,Euro,ACH,1,CYCLE: Max 9 hops
137931,2022/11/08 09:12,1538,8008279E0,12883,801345490,5698.52,Euro,5698.52,Euro,ACH,1,Out
137932,2022/11/20 18:34,1538,8008279E0,1226,8001672F0,6043.82,Euro,6043.82,Euro,ACH,1,Out
137933,2022/11/25 11:27,1538,8008279E0,1213,8006E6420,9483.76,Euro,9483.76,Euro,ACH,1,Out


In [9]:
currency_summary = patterns_large.groupby('Receiving Currency')['Amount Received'].sum()
currency_summary

Receiving Currency
Australian Dollar    7.584440e+09
Bitcoin              1.188709e+01
Brazil Real          2.168496e+10
Canadian Dollar      2.866334e+09
Euro                 2.295050e+11
Mexican Peso         8.778921e+08
Ruble                1.701728e+12
Rupee                5.856376e+12
Saudi Riyal          4.604570e+07
Shekel               7.417693e+10
Swiss Franc          2.120947e+08
UK Pound             9.846324e+09
US Dollar            2.327460e+11
Yen                  2.497251e+12
Yuan                 2.620874e+11
Name: Amount Received, dtype: float64

In [ ]:
anomalies = patterns_large[patterns_large['Is Laundering'] == 1]
anomalies


,Timestamp,From Bank,Account,To Bank,Account_1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Anomaly Type
0,2022/08/09 05:14,952,8139F54E0,111632,8062C56E0,5331.44,US Dollar,5331.44,US Dollar,ACH,1,STACK
1,2022/08/13 13:09,111632,8062C56E0,8456,81363F620,5602.59,US Dollar,5602.59,US Dollar,ACH,1,STACK
2,2022/08/15 07:40,118693,823D5EB90,13729,801CF2E60,1400.54,US Dollar,1400.54,US Dollar,ACH,1,STACK
3,2022/08/15 14:19,13729,801CF2E60,123621,81A7090F0,1467.94,US Dollar,1467.94,US Dollar,ACH,1,STACK
4,2022/08/13 12:40,24750,81363F410,213834,808757B00,16898.29,US Dollar,16898.29,US Dollar,ACH,1,STACK
...,...,...,...,...,...,...,...,...,...,...,...,...
137931,2022/11/08 09:12,1538,8008279E0,12883,801345490,5698.52,Euro,5698.52,Euro,ACH,1,Out
137932,2022/11/20 18:34,1538,8008279E0,1226,8001672F0,6043.82,Euro,6043.82,Euro,ACH,1,Out
137933,2022/11/25 11:27,1538,8008279E0,1213,8006E6420,9483.76,Euro,9483.76,Euro,ACH,1,Out
137934,2022/11/28 09:15,1538,8008279E0,1827,80117E140,11919.51,Euro,11919.51,Euro,ACH,1,Out


In [ ]:
sorted_df = patterns_large.sort_values('Amount Received', ascending=False)
sorted_df


,Timestamp,From Bank,Account,To Bank,Account_1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Anomaly Type
52149,2022/09/17 04:02,18184,80F5A2240,4,816071A90,5.257959e+12,Rupee,5.257959e+12,Rupee,ACH,1,STACK
67435,2022/09/23 09:49,36897,82223D560,239767,812CC8BC0,1.055737e+12,Yen,1.055737e+12,Yen,ACH,1,CYCLE: Max 2 hops
24,2022/08/19 17:43,266946,81A2D04C0,266946,81A4AFE20,6.759616e+11,Ruble,6.759616e+11,Ruble,ACH,1,In
24887,2022/08/30 13:16,23351,805E42F30,251758,814C098C0,2.928976e+11,Yen,2.928976e+11,Yen,ACH,1,BIPARTITE
8921,2022/08/14 16:33,25935,82170BFC0,137768,812CEEB40,2.430187e+11,Yen,2.430187e+11,Yen,ACH,1,CYCLE: Max 9 hops
...,...,...,...,...,...,...,...,...,...,...,...,...
72101,2022/10/05 15:51,0,80010F381,17,8000FC641,1.561410e-01,Bitcoin,1.561410e-01,Bitcoin,Bitcoin,1,STACK
48786,2022/11/10 09:42,126831,80AA5E731,123,80006B361,1.471310e-01,Bitcoin,1.471310e-01,Bitcoin,Bitcoin,1,In
46288,2022/09/16 05:32,0,8024A3800,228650,814853040,1.200000e-01,Euro,1.200000e-01,Euro,ACH,1,BIPARTITE
27859,2022/08/26 09:06,82071,824D68DD0,2105779,826BF6060,9.000000e-02,UK Pound,9.000000e-02,UK Pound,ACH,1,In


In [17]:
high_value_transactions = patterns_large[patterns_large['Amount Received'] > 10000]
high_value_transactions

,Timestamp,From Bank,Account,To Bank,Account_1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Anomaly Type
4,2022/08/13 12:40,24750,81363F410,213834,808757B00,16898.29,US Dollar,16898.29,US Dollar,ACH,1,STACK
5,2022/08/22 06:34,213834,808757B00,0,800073EF0,17607.19,US Dollar,17607.19,US Dollar,ACH,1,STACK
6,2022/08/01 00:19,134266,814167590,36925,810E343A0,132713.46,Yuan,132713.46,Yuan,ACH,1,CYCLE: Max 12 hops
7,2022/08/01 13:05,36925,810E343A0,119211,814AB4F60,18264.20,US Dollar,18264.20,US Dollar,ACH,1,CYCLE: Max 12 hops
8,2022/08/03 13:28,119211,814AB4F60,132965,81B88A230,14567.69,Euro,14567.69,Euro,ACH,1,CYCLE: Max 12 hops
...,...,...,...,...,...,...,...,...,...,...,...,...
137927,2022/11/27 12:19,8456,80999EE00,239461,81047FC40,23995.70,Yuan,23995.70,Yuan,ACH,1,CYCLE: Max 9 hops
137929,2022/12/02 15:12,224391,816047D00,21582,80CCC9710,21214.64,Yuan,21214.64,Yuan,ACH,1,CYCLE: Max 9 hops
137930,2022/11/05 23:59,1538,8008279E0,23435,8017863F0,16425.62,US Dollar,16425.62,US Dollar,ACH,1,Out
137934,2022/11/28 09:15,1538,8008279E0,1827,80117E140,11919.51,Euro,11919.51,Euro,ACH,1,Out


In [2]:
file_path = "../data/darkness-dataset.txt"
with open(file_path, 'r', encoding='utf-8') as file:
    for _ in range(10):
        print(file.readline())


id,name,country_code,position,city,current_company,about,posts,experience,url,people_also_viewed,educations_details,education,recommendations_count,avatar,languages,certifications,recommendations,volunteer_experience,courses,followers,connections,current_company_company_id,current_company_name,publications,patents,projects,organizations,input_url,linkedin_id;;;;

"random-id-2468,""Random Name"",""KW"","""",""Kuwait"",""{""""company_id"""":""""-seara"""",""""industry"""":""""Seara Alimentos LTDA"""",""""link"""":""""https://br.linkedin.com/company/-seara?trk=public_profile_topcard-current-company"""",""""name"""":""""Random Alimentos LTDA""""}"",""Experienced Unit Sales Manager with a demonstrated history of working in the foodâ€¦"",""[{""""attribution"""":""""By Random Name"""",""""created_at"""":""""2018-09-05T00:00:00.000Z"""",""""img"""":""""https://static.licdn.com/scds/common/u/img/pic/pic_pulse_stock_article_1.jpg"""",""""interaction"""":""""By Random Name"""",""""link"""":""""ht

In [3]:
#df = pd.read_csv(file_path, delimiter=";", encoding="utf-8") 
df = pd.read_csv(file_path, delimiter=",", encoding="utf-8", quotechar='"', on_bad_lines='skip')
df

#df.columns

,id,name,country_code,position,city,current_company,about,posts,experience,url,...,followers,connections,current_company_company_id,current_company_name,publications,patents,projects,organizations,input_url,linkedin_id;;;;
0,"random-id-2468,""Random Name"",""KW"","""",""Kuwait"",...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"random-id-1357,""Random Name"",""IN"",""Random EduT...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"random-id-8642,""Random Name"","""","""","""",""{""""comp...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"random-id-9512,""Random Name"",""UK"",""Design Engi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"random-id-8642,""Random Name"",""KW"","""",""Kuwait"",...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"random-id-7531,""Random Name"",""KW"","""",""Kuwait"",...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"random-id-4820,""Random Name"",""NG"",""Volkswagen ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"random-id-9025,""Random Name"",""UK"","""","""",""{""""co...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"random-id-5837,""Random Name"",""EG"",""Food Safety...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.loc[8, 'id']

'random-id-5837,"Random Name","EG","Food Safety & hygiene coordinator at Random Hotels Group & Random Alcazar South Sinai","Cairo","{""company_id"":""steigenberger"",""industry"":""Steigenberger Hotels and Resorts"",""link"":""https://de.linkedin.com/company/steigenberger?trk=public_profile_topcard-current-company"",""name"":""Random Hotels and Resorts"",""title"":""Food Safety & hygiene coordinator at Random Hotels Group & Random Alcazar South Sinai""}","","[]","[{""description"":""Ø¬Ù†ÙˆØ¨ Ø³ÙŠÙ†Ø§Ø¡ Ø´Ø±Ù… Ø§Ù„Ø´ÙŠØ® Ù…ØµØ±"",""description_html"":null,""duration"":""Oct 2023 8 months"",""end_date"":""8 months"",""start_date"":""Oct 2023"",""subtitle"":""Random Hotels and Resorts"",""subtitleURL"":""https://de.linkedin.com/company/steigenberger?trk=public_profile_experience-item_profile-section-card_subtitle-click"",""title"":""Food safety and hygiene coordinator""},{""description"":""Ù…ØµØ±"",""description_html"":null,""duration"":""Mar 2023 Oct 2023 8 months"",""duration_short"":""

In [13]:
def funct(row):
    print(row)
    
df.apply(funct,axis=1)

id                            random-id-2468,"Random Name","KW","","Kuwait",...
name                                                                        NaN
country_code                                                                NaN
position                                                                    NaN
city                                                                        NaN
current_company                                                             NaN
about                                                                       NaN
posts                                                                       NaN
experience                                                                  NaN
url                                                                         NaN
people_also_viewed                                                          NaN
educations_details                                                          NaN
education                               

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
dtype: object

In [6]:
def cleaned_columns(df, column_name):
    def process_cleaning_rows(input_str):
        input_str = input_str.replace('""', '"')
        json_removed = re.sub(r'(\{|\[|]|\})', ' ', str(input_str))
        cleaned_text = re.sub(r'["\\|]', ' ', json_removed)
        cleaned_text = re.sub(r'[^\w\s:/\.\-?&=]', ' ', cleaned_text)
        final_cleaned = re.sub(r'\s+', ' ', cleaned_text).strip()
        return final_cleaned
    df[f'{column_name}_cleaned'] = df[column_name].apply(process_cleaning_rows)
    return df
df_cleaned1 = cleaned_columns(df, 'id')


In [7]:
df_cleaned1.loc[1, 'id_cleaned']


'random-id-2468 Random Name KW Kuwait company_id : -seara industry : Seara Alimentos LTDA link : https://br.linkedin.com/company/-seara?trk=public_profile_topcard-current-company name : Random Alimentos LTDA Experienced Unit Sales Manager with a demonstrated history of working in the foodâ attribution : By Random Name created_at : 2018-09-05T00:00:00.000Z img : https://static.licdn.com/scds/common/u/img/pic/pic_pulse_stock_article_1.jpg interaction : By Random Name link : https://www.linkedin.com/pulse/hacking-linkedin-account-ahmed-nagib?trk=public_profile_article_view title : Hacking LinkedIn Account https://www.linkedin.com/in/random-id-2468 profile_link : https://kw.linkedin.com/in/moamen-yousef-147392136?trk=public_profile_browsemap-profile profile_link : https://kw.linkedin.com/in/mohamed-mahmoud-zaki-nakhla-24009557?trk=public_profile_browsemap-profile profile_link : https://kw.linkedin.com/in/yasser-yassin-46758626?trk=public_profile_browsemap-profile profile_link : https://kw.

In [56]:
df_cleaned = df_cleaned1[['id', 'id_cleaned']]


In [57]:
df_cleaned

,id,id_cleaned
0,"random-id-2468,""Random Name"",""KW"","""",""Kuwait"",...",random-id-2468 Random Name KW Kuwait company_i...
1,"random-id-1357,""Random Name"",""IN"",""Random EduT...",random-id-1357 Random Name IN Random EduTech N...
2,"random-id-8642,""Random Name"","""","""","""",""{""""comp...",random-id-8642 Random Name company_id : shepar...
3,"random-id-9512,""Random Name"",""UK"",""Design Engi...",random-id-9512 Random Name UK Design Engineer ...
4,"random-id-8642,""Random Name"",""KW"","""",""Kuwait"",...",random-id-8642 Random Name KW Kuwait company_i...
5,"random-id-7531,""Random Name"",""KW"","""",""Kuwait"",...",random-id-7531 Random Name KW Kuwait company_i...
6,"random-id-4820,""Random Name"",""NG"",""Volkswagen ...",random-id-4820 Random Name NG Volkswagen Produ...
7,"random-id-9025,""Random Name"",""UK"","""","""",""{""""co...",random-id-9025 Random Name UK company_id : old...
8,"random-id-5837,""Random Name"",""EG"",""Food Safety...",random-id-5837 Random Name EG Food Safety & hy...


In [10]:
df_cleaned.loc[0, 'id_cleaned']

'random-id-2468 Random Name KW Kuwait company_id : -seara industry : Seara Alimentos LTDA link : https://br.linkedin.com/company/-seara?trk=public_profile_topcard-current-company name : Random Alimentos LTDA Experienced Unit Sales Manager with a demonstrated history of working in the foodâ attribution : By Random Name created_at : 2018-09-05T00:00:00.000Z img : https://static.licdn.com/scds/common/u/img/pic/pic_pulse_stock_article_1.jpg interaction : By Random Name link : https://www.linkedin.com/pulse/hacking-linkedin-account-ahmed-nagib?trk=public_profile_article_view title : Hacking LinkedIn Account https://www.linkedin.com/in/random-id-2468 profile_link : https://kw.linkedin.com/in/moamen-yousef-147392136?trk=public_profile_browsemap-profile profile_link : https://kw.linkedin.com/in/mohamed-mahmoud-zaki-nakhla-24009557?trk=public_profile_browsemap-profile profile_link : https://kw.linkedin.com/in/yasser-yassin-46758626?trk=public_profile_browsemap-profile profile_link : https://kw.

In [ ]:
id Name country Ab_country  country  company_id industry link name  job title descr profile_link  description_html University duration duration_short end_date start_date subtitle subtitleURL

In [28]:
def extract_columns(df, cleaned_column):
    def extract_id(text):
        match = re.search(r'\bid-(\d+)\b', text)
        return f"id-{match.group(1)}" if match else None

    def extract_abbreviations(text):
        match = re.findall(r'\b[A-Z]{2}\b', text)
        return " ".join(match) if match else None

    df['id'] = df[cleaned_column].apply(extract_id)
    df['abbreviations'] = df[cleaned_column].apply(extract_abbreviations)
    return df
df_result = extract_columns(df, 'id_cleaned')


In [30]:
def extract_data(row):
    data = {
        # ID: ищем шаблон "id-число" (например, id-5837)
        'id': re.search(r'\bid-\d+\b', row),
        
        # Name: предположим, это первое слово с большой буквы (или несколько слов)
        'Name': re.search(r'\b[A-Z][a-zA-Z]+\s[A-Z][a-zA-Z]+\b', row),
        
        # Country abbreviations: два символа в верхнем регистре
        'abbreviations': re.search(r'\b[A-Z]{2}\b', row),
        
        # Country: находим страну (например, "Kuwait", "USA")
        'country': re.search(r'\b[A-Z][a-z]+\b', row),
        
        # Company ID: текст рядом с "company_id"
        'company_id': re.search(r'company_id :\s(\S+)', row),
        
        # Industry: текст рядом с "industry"
        'industry': re.search(r'industry\s([A-Za-z\s]+)', row),
        
        # Link: любая ссылка
        'link': re.search(r'https?://\S+', row),
        
        # Job Title: ищем текст рядом с "title"
        'job title': re.search(r'title :\s([A-Za-z\s\-]+)', row),
        
        # Description: текст рядом с "descr" (например, описание работы)
        'descr': re.search(r'descr :\s([A-Za-z\s\-]+)', row),
        
        # Profile Link: ищем ссылки на профили
        'profile_link': re.search(r'profile_link :\shttps?://\S+', row),
        
        # University: текст рядом с "University"
        'University': re.search(r'University\s([A-Za-z\s]+)', row),
        
        # Duration: ищем текст рядом с "duration"
        'duration': re.search(r'duration :\s([A-Za-z0-9\s\-]+)', row),
        
        # Subtitle: текст рядом с "subtitle"
        'subtitle': re.search(r'subtitle :\s([A-Za-z\s]+)', row),
        
        # Subtitle URL: текст рядом с "subtitleURL"
        'subtitleURL': re.search(r'subtitleURL :\shttps?://\S+', row),
        
        # Dates (start_date и end_date)
        'start_date': re.search(r'start_date :\s(\w+\s\d+)', row),
        'end_date': re.search(r'end_date :\s(\w+\s\d+)', row)
    }
    
    # Извлекаем данные из объектов Match и возвращаем их в виде строки
    return {key: match.group(0) if match else None for key, match in data.items()}

# Применяем функцию к колонке id_cleaned
columns = [
    'id', 'Name', 'abbreviations', 'country', 'company_id', 'industry',
    'link', 'job title', 'descr', 'profile_link', 'University',
    'duration', 'subtitle', 'subtitleURL', 'start_date', 'end_date'
]

# Предположим, `df_cleaned` содержит колонку `id_cleaned` с очищенными строками
df_extracted = df_cleaned['id_cleaned'].apply(extract_data).apply(pd.Series)

# Добавляем новые колонки в DataFrame
df_cleaned = pd.concat([df_cleaned, df_extracted[columns]], axis=1)

In [35]:
df_extracted

,id,Name,abbreviations,country,company_id,industry,link,job title,descr,profile_link,University,duration,subtitle,subtitleURL,start_date,end_date
0,id-2468,Random Name,KW,Random,company_id : -seara,None,https://br.linkedin.com/company/-seara?trk=pub...,title : Hacking LinkedIn Account https,None,profile_link : https://kw.linkedin.com/in/moam...,University https,None,subtitle : Professional working proficiency ti...,None,None,None
1,id-1357,Random Name,IN,Random,company_id : byjus,None,https://in.linkedin.com/company/byjus?trk=publ...,title : Random EduTech company,None,profile_link : https://in.linkedin.com/in/anan...,University degree,duration : Jun 2021 - Present 3 years 1 month ...,None,None,start_date : Jun 2021,end_date : May 2021
2,id-8642,Random Name,None,Random,company_id : shepard-exposition-services,None,https://www.linkedin.com/company/shepard-expos...,None,None,None,None,None,None,None,None,None
3,id-9512,Random Name,UK,Random,company_id : edf-energy,None,https://uk.linkedin.com/company/edf-energy?trk...,title : Design Engineer System health at Rando...,None,None,None,duration : Aug 2021 2 years 10 months end,subtitle : EDF UK subtitleURL,subtitleURL : https://uk.linkedin.com/company/...,start_date : Aug 2021,end_date : Aug 2021
4,id-8642,Random Name,KW,Random,company_id : global-infinity,None,https://kw.linkedin.com/company/global-infinit...,title : Professional working proficiency title,None,profile_link : https://kw.linkedin.com/in/maze...,University https,None,subtitle : Professional working proficiency ti...,None,None,None
5,id-7531,Random Name,KW,Random,company_id : global-infinity,None,https://kw.linkedin.com/company/global-infinit...,title : Professional working proficiency title,None,profile_link : https://kw.linkedin.com/in/maze...,University https,None,subtitle : Professional working proficiency ti...,None,None,None
6,id-4820,Random Name,NG,Random,None,None,https://m.facebook.com/profile.php?id=72082108...,title : Volkswagen Product at Random Auto Sale...,None,profile_link : https://ng.linkedin.com/in/taiw...,None,None,None,None,None,None
7,id-9025,Random Name,UK,Random,company_id : oldmill,None,https://uk.linkedin.com/company/oldmill?trk=pu...,None,None,None,University of Exeter https,None,None,None,None,None
8,id-5837,Random Name,EG,Random,company_id : steigenberger,None,https://de.linkedin.com/company/steigenberger?...,title : Food Safety,None,None,University degree,duration : Oct 2023 8 months end,subtitle : Random Hotels and Resorts subtitleURL,subtitleURL : https://de.linkedin.com/company/...,start_date : Oct 2023,end_date : Oct 2023


In [56]:
def remove_exact_prefixes(df_extracted):
    for column in df_extracted.columns:
        df_extracted[column] = df_extracted[column].apply(
            lambda x: x[len(column) + 2:].strip() if isinstance(x, str) and x.startswith(f"{column} :") else x
        )
    return df_extracted

df_extracted = remove_exact_prefixes(df_extracted)

print(df_extracted.head())

        id         Name abbreviations country                   company_id  \
0  id-2468  Random Name            KW  Random                       -seara   
1  id-1357  Random Name            IN  Random                        byjus   
2  id-8642  Random Name          None  Random  shepard-exposition-services   
3  id-9512  Random Name            UK  Random                   edf-energy   
4  id-8642  Random Name            KW  Random              global-infinity   

  industry                                               link  \
0     None  https://br.linkedin.com/company/-seara?trk=pub...   
1     None  https://in.linkedin.com/company/byjus?trk=publ...   
2     None  https://www.linkedin.com/company/shepard-expos...   
3     None  https://uk.linkedin.com/company/edf-energy?trk...   
4     None  https://kw.linkedin.com/company/global-infinit...   

                                           job title descr  \
0             title : Hacking LinkedIn Account https  None   
1               

In [57]:
df_extracted

,id,Name,abbreviations,country,company_id,industry,link,job title,descr,profile_link,University,duration,subtitle,subtitleURL,start_date,end_date
0,id-2468,Random Name,KW,Random,-seara,None,https://br.linkedin.com/company/-seara?trk=pub...,title : Hacking LinkedIn Account https,None,https://kw.linkedin.com/in/moamen-yousef-14739...,University https,None,Professional working proficiency title,None,None,None
1,id-1357,Random Name,IN,Random,byjus,None,https://in.linkedin.com/company/byjus?trk=publ...,title : Random EduTech company,None,https://in.linkedin.com/in/anand-kumar-pandey-...,University degree,Jun 2021 - Present 3 years 1 month duration,None,None,Jun 2021,May 2021
2,id-8642,Random Name,None,Random,shepard-exposition-services,None,https://www.linkedin.com/company/shepard-expos...,None,None,None,None,None,None,None,None,None
3,id-9512,Random Name,UK,Random,edf-energy,None,https://uk.linkedin.com/company/edf-energy?trk...,title : Design Engineer System health at Rando...,None,None,None,Aug 2021 2 years 10 months end,EDF UK subtitleURL,https://uk.linkedin.com/company/edf-energy?trk...,Aug 2021,Aug 2021
4,id-8642,Random Name,KW,Random,global-infinity,None,https://kw.linkedin.com/company/global-infinit...,title : Professional working proficiency title,None,https://kw.linkedin.com/in/mazen-sahyouni-b281...,University https,None,Professional working proficiency title,None,None,None
5,id-7531,Random Name,KW,Random,global-infinity,None,https://kw.linkedin.com/company/global-infinit...,title : Professional working proficiency title,None,https://kw.linkedin.com/in/mazen-sahyouni-b281...,University https,None,Professional working proficiency title,None,None,None
6,id-4820,Random Name,NG,Random,None,None,https://m.facebook.com/profile.php?id=72082108...,title : Volkswagen Product at Random Auto Sale...,None,https://ng.linkedin.com/in/taiwo-daniel-aa9b15...,None,None,None,None,None,None
7,id-9025,Random Name,UK,Random,oldmill,None,https://uk.linkedin.com/company/oldmill?trk=pu...,None,None,None,University of Exeter https,None,None,None,None,None
8,id-5837,Random Name,EG,Random,steigenberger,None,https://de.linkedin.com/company/steigenberger?...,title : Food Safety,None,None,University degree,Oct 2023 8 months end,Random Hotels and Resorts subtitleURL,https://de.linkedin.com/company/steigenberger?...,Oct 2023,Oct 2023


In [58]:
df_extracted['job title']

0               title : Hacking LinkedIn Account https
1                      title : Random EduTech company 
2                                                 None
3    title : Design Engineer System health at Rando...
4      title : Professional working proficiency title 
5      title : Professional working proficiency title 
6    title : Volkswagen Product at Random Auto Sale...
7                                                 None
8                                 title : Food Safety 
Name: job title, dtype: object

In [59]:
df_extracted['job title'] = df_extracted['job title'].apply(
    lambda x: x.replace('title :', '').strip() if isinstance(x, str) else x
)

In [65]:
df_extracted.columns
df_extracted = df_extracted.fillna('')


In [66]:
client.insert('card_data.darknet_data', df_extracted)

In [17]:
text = '[{"apple": "value1"},{"b": "va"}],[{"apple": "value1"},{"b": "va"}]'
result  = re.findall(r'\[{.*}\]',  text)
print(result)

['[{"apple": "value1"},{"b": "va"}],[{"apple": "value1"},{"b": "va"}]']
